# Light-rail Rapid Transit Report

# Note: This is a work-in-progress as of 25 October 2021.
#            It is under development, and NOT for general use!

This notebook generates a single summary report on the total boardings by station for all MBTA light rail rapid transit routes, i.e., the Green Line (Branches B, C, D, and E) and the Mattapan Trolley. 

This notebook does _not_ produce a visualization of the report results.
Use the __TBD_X__ notebook for this purpose.

To generate a report comparing the total boardings for all heavy rail rapid transit routes 
under __two__ scenarios:
* Run this notebook for the first scenario, capturing the output in a specified CSV file.
* Run this notebook for the _second_ scenario, storing the output in a _second_ CSV file.
* Run the notebook __TBDY__ to generate a comparative report of the two scenarios.

In [ ]:
# Import required packages
import pandas as pd
import numpy as np
import os
import glob
from functools import reduce

###  User input required: Specify (and run) config.py file - get names of input and output directories

In [ ]:
%run "S:/jupyter_notebooks/config.py"

### User input required: Specify scenario to run, referencing relevant variable in config.py file

In [ ]:
scenario = base_scenario_dir

### User input required: Specify 'suffix' (after scenario name, before '.csv' suffix)
The full output CSV file name consists of:
1. The path to your "sandbox directory", specified in config.py
2. The scenario name (with blanks replaced by underscores)
3. The filename 'suffix' you specify in the next cell
4. '.csv'

In [ ]:
output_csv_fn_suffix = 'light_rail_rt_report_25Oct2021'

### Debug/trace flag: Principally for use during development, testing, etc.

In [ ]:
debug_trace_flag = False

In [ ]:
def dump_csv(df, filename):
    df.to_csv(sandbox_dir + filename)

### Beginning of code for "library" routines - to be moved into modxlib.

In [ ]:
# calculate_total_daily_boardings: Calculate the daily total across all time periods.
#
#    This calculation requires a bit of subtelty, because the number of rows in the four
#    data frames produced by produced in the calling function is NOT necessarily the same. 
#    A brute-force apporach will not work, generally speaking.
#    See comments in the code below for details.
#
# NOTE: This is a helper function for import_transit_assignment (q.v.)
#   
# Parameter: boardings_by_tod - a dict with the keys 'AM', 'MD', 'PM', and 'NT'
#            for which the value of each key is a data frame containing the total
#            boardings for the list of routes specified in the input CSV file.
#
# Return value: The input dict (boardings_by_tod) with an additional key 'daily'
#               the value of which is a dataframe with the total daily boardings
#               for all routes specified in the input CSV across all 4 time periods.
#
def calculate_total_daily_boardings(boardings_by_tod):
    am_results = boardings_by_tod['AM']
    md_results = boardings_by_tod['MD']
    pm_results = boardings_by_tod['PM']
    nt_results = boardings_by_tod['NT']
    
    # Compute the daily sums.
    #
    # Step 1: Join 'am' and 'md' dataframes
    j1 = pd.merge(am_results, md_results, on=['ROUTE', 'STOP'], how='outer', suffixes=('_am', '_md'))
    # Step 1.1 Replace NaN's with 0's
    j1 = j1.fillna(0)

    # Step 1.2 Compute the 'AM' + 'MD' sums
    j1['DirectTransferOff'] = j1['DirectTransferOff_am'] + j1['DirectTransferOff_md']
    j1['DirectTransferOn'] = j1['DirectTransferOn_am'] + j1['DirectTransferOn_md']
    j1['DriveAccessOn'] = j1['DriveAccessOn_am'] + j1['DriveAccessOn_md']
    j1['EgressOff'] = j1['EgressOff_am'] + j1['EgressOff_md']
    j1['Off'] = j1['Off_am'] + j1['Off_md']
    j1['On'] = j1['On_am'] + j1['On_md']
    j1['WalkAccessOn'] = j1['WalkAccessOn_am'] + j1['WalkAccessOn_md'] 
    j1['WalkTransferOff'] = j1['WalkTransferOff_am'] + j1['WalkTransferOff_md']
    j1['WalkTransferOn'] = j1['WalkTransferOn_am'] + j1['WalkTransferOn_md']

    # Step 1.3: Drop un-needed columns
    cols_to_drop = ['DirectTransferOff_am', 'DirectTransferOff_md',
                    'DirectTransferOn_am', 'DirectTransferOn_md',
                    'DriveAccessOn_am', 'DriveAccessOn_md',
                    'EgressOff_am','EgressOff_md',
                    'Off_am', 'Off_md',
                    'On_am', 'On_md',
                    'WalkAccessOn_am', 'WalkAccessOn_md',
                    'WalkTransferOff_am', 'WalkTransferOff_md',
                    'WalkTransferOn_am', 'WalkTransferOn_md'
                    ]
    j1 = j1.drop(columns=cols_to_drop)

    # Step 2: j2 - join 'pm' and 'nt' data frames
    j2 = pd.merge(pm_results, nt_results, on=['ROUTE', 'STOP'], how='outer', suffixes=('_pm', '_nt'))
    # Step 2.1: Replace NaN's with 0's
    j2 = j2.fillna(0)

    # Step 2.2: Compute the 'PM' + 'NT' sums
    j2['DirectTransferOff'] = j2['DirectTransferOff_pm'] + j2['DirectTransferOff_nt']
    j2['DirectTransferOn'] = j2['DirectTransferOn_pm'] + j2['DirectTransferOn_nt']
    j2['DriveAccessOn'] = j2['DriveAccessOn_pm'] + j2['DriveAccessOn_nt']
    j2['EgressOff'] = j2['EgressOff_pm'] + j2['EgressOff_nt']
    j2['Off'] = j2['Off_pm'] + j2['Off_nt']
    j2['On'] = j2['On_pm'] + j2['On_nt']
    j2['WalkAccessOn'] = j2['WalkAccessOn_pm'] + j2['WalkAccessOn_nt'] 
    j2['WalkTransferOff'] = j2['WalkTransferOff_pm'] + j2['WalkTransferOff_nt']
    j2['WalkTransferOn'] = j2['WalkTransferOn_pm'] + j2['WalkTransferOn_nt']

    # Step 2.3: Drop un-needed columns
    cols_to_drop = ['DirectTransferOff_pm', 'DirectTransferOff_nt',
                    'DirectTransferOn_pm', 'DirectTransferOn_nt',
                    'DriveAccessOn_pm', 'DriveAccessOn_nt',
                    'EgressOff_pm','EgressOff_nt',
                    'Off_pm', 'Off_nt',
                    'On_pm', 'On_nt',
                    'WalkAccessOn_pm', 'WalkAccessOn_nt',
                    'WalkTransferOff_pm', 'WalkTransferOff_nt',
                    'WalkTransferOn_pm', 'WalkTransferOn_nt'
                    ]
    j2 = j2.drop(columns=cols_to_drop)

    # Step 3: Join "j1" and "j2" to produce a dataframe with the daily totals
    daily_df = pd.merge(j1, j2, on=['ROUTE', 'STOP'], how='outer', suffixes=('_j1', '_j2'))
    # Step 3.1 : Replace any NaN's with 0's. This line _shouldn't_ be needed - just being extra cautious.
    daily_df = daily_df.fillna(0)

    # Step 3.2 : Compute THE daily sums
    daily_df['DirectTransferOff'] = daily_df['DirectTransferOff_j1'] + daily_df['DirectTransferOff_j2']
    daily_df['DirectTransferOn'] = daily_df['DirectTransferOn_j1'] + daily_df['DirectTransferOn_j2']
    daily_df['DriveAccessOn'] = daily_df['DriveAccessOn_j1'] + daily_df['DriveAccessOn_j2']
    daily_df['EgressOff'] = daily_df['EgressOff_j1'] + daily_df['EgressOff_j2']
    daily_df['Off'] = daily_df['Off_j1'] + daily_df['Off_j2']
    daily_df['On'] = daily_df['On_j1'] + daily_df['On_j2']
    daily_df['WalkAccessOn'] = daily_df['WalkAccessOn_j1'] + daily_df['WalkAccessOn_j2'] 
    daily_df['WalkTransferOff'] = daily_df['WalkTransferOff_j1'] + daily_df['WalkTransferOff_j2']
    daily_df['WalkTransferOn'] = daily_df['WalkTransferOn_j1'] + daily_df['WalkTransferOn_j2']

    # Step 3.3 : Drop un-needed columns
    cols_to_drop = ['DirectTransferOff_j1', 'DirectTransferOff_j2',
                    'DirectTransferOn_j1', 'DirectTransferOn_j2',
                    'DriveAccessOn_j1', 'DriveAccessOn_j2',
                    'EgressOff_j1','EgressOff_j2',
                    'Off_j1', 'Off_j2',
                    'On_j1', 'On_j2',
                    'WalkAccessOn_j1', 'WalkAccessOn_j2',
                    'WalkTransferOff_j1', 'WalkTransferOff_j2',
                    'WalkTransferOn_j1', 'WalkTransferOn_j2'
                    ]
    daily_df = daily_df.drop(columns=cols_to_drop)

    # Finally, we've got the 'daily' total dataframe!
    boardings_by_tod['daily'] = daily_df
    return boardings_by_tod
# end_def calculate_total_daily_boardings()

In [ ]:
# import_transit_assignment: Import transit assignment result CSV files for a given scenario.
#
# 1. Read all CSV files for each time period ('tod'), and caclculate the sums for each time period.
#    Step 1 can be performed as a brute-force sum across all columns, since the number of rows in
#    the CSVs (and thus the dataframes) for any given time period are all the same.
#
# 2. Calculate the daily total across all time periods.
#    Step 2 requires a bit of subtelty, because the number of rows in the data frames produced in 
#    Step 1 is NOT necessarily the same. A brute-force apporach will not work, generally speaking.
#    See comments in the code below for details.
#    NOTE: This step is performed by the helper function calculate_total_daily_boardings.
#
# 3. Return value: a dict of the form:
#    {'AM'    : dataframe with totals for the AM period,
#     'MD'    : datafrme with totals for the MD period,
#     'PM'    : dataframe with totals for the PM period,
#     'NT'    : dataframe with totals for the NT period,
#     'daily' : dataframe with totals for the entire day
#   }
# 
def import_transit_assignment(scenario):
    base = scenario + r'out/'
    tods = ["AM", "MD", "PM", "NT"]
    # At the end of execution of this function, the dictionary variable'TODsums' will contain all the TOD summed results:
    # one key-value-pair for each 'tod' AND the 'daily' total as well.
    
    # The dict 'TODsums' is the return value of this function.
    TODsums = { 'AM' : None, 'MD' : None, 'PM' : None, 'NT' : None }

    # Import CSV files and create sum tables for each T-O-D (a.k.a. 'time period').
    for tod in tods:
        # Get full paths to _all_ CSV files for the current t-o-d.
        x = tod + '/' 
        fq_csv_fns = glob.glob(os.path.join(base,x,r'*.csv'))
        
        # 'tablist' : List of all the dataframes created from reading in the all the CSV files for the current t-o-d
        tablist = []
        for csv_file in fq_csv_fns:
            # Read CSV file into dataframe, set indices, and append to 'tablist'
            tablist.append(pd.read_csv(csv_file).set_index(['ROUTE','STOP']))
        #
  
        # Sum the tables for the current TOD
        TODsums[tod] = reduce(lambda a, b: a.add(b, fill_value=0), tablist)
    # end_for over all tod's

    TODsums =  calculate_total_daily_boardings(TODsums)
    
    # Ensure that the ROUTE and STOP columns of each dataframe in TODsums aren't indices.
    for k in TODsums.keys():
        TODsums[k] = TODsums[k].reset_index()
    #
    return TODsums
# end_def import_transit_assignment()

In [ ]:
# First crack at more meaningful route classification for transit report.
# WARNING / NOTE: This classification is specific to TDM19 and will be replaced for TDM23!!!
#
# Define data structure and function to classify transit routes for reporting purposes.
_classification_table = {
    1:  'MBTA Bus - Local Bus',
    2:  'MBTA Bus - Inner Express Bus',
    3:  'MBTA Bus - Outer Express Bus' ,
    4:  'Green Line',
    5:  'Red Line',
    6:  'Mattapan Trolley',
    7:  'Orange Line',
    8:  'Blue Line',
    9:  'Commuter Rail - Fairmount Line',
    10: 'Ferries - Inner Harbor',
    11: 'Ferries - Outer Harbor',
    12: 'Silver Line',
    13: 'Sliver Line',
    14: 'Logan Express',
    15: 'Logan Shuttle',
    16: 'MGH and Other Shuttles',
    17: 'RTA Bus - Brockton RTA',
    18: 'RTA Bus - CATA RTA',
    19: 'RTA Bus - GATRA RTA',
    20: 'RTA Bus - Lowell RTA',
    21: 'RTA Bus - Merrimack RTA',
    22: 'RTA Bus - MetroWest RTA',
    23: 'Private Bus - Bloom',
    24: 'Private Bus - C & J Bus',
    25: 'Private Bus - Cavalier Bus',
    26: 'Private Bus - Concord Coach',
    27: 'Private Bus - Dattco Bus',
    28: 'Private Bus - Plymouth & Brockton',
    29: 'Private Bus - Peter Pan',
    30: 'Private Bus - Yankee',
    31: 'MBTA Subsidized Bus Routes',
    32: 'Commuter Rail - Beverly / Newburyport / Rockport Line',
    33: 'Commuter Rail - Stoughton / Providence Line',
    34: 'Commuter Rail - Greenbush / Plymouth / Kingston / Middleborough Line',
    35: 'Commuter Rail - Haverhill Line',
    36: 'Commuter Rail - Lowell Line',
    37: 'Commuter Rail - Fitchburg Line',
    38: 'Commuter Rail - Framingham / Worcester Line',
    39: 'Commuter Rail - Needham Line',
    40: 'Commuter Rail - Franklin Line',
    41: 'RTA Bus - SRTA RTA',
    42: 'RTA Bus - Worcester RTA',
    43: 'RTA Bus- Pioneer Valley RTA',
    70: 'Walk' }

def classify_green_line_route(row):
    # Crude, first-crack implementation based on TDM19 'Routes_ID' field.
    retval = 'Rapid Transit - Green Line '
    route_id = row['Routes_ID']
    if route_id in [6034, 6035]:
        retval += 'B Branch'
    elif route_id in [6032, 6033]:
        retval += 'C Branch'
    elif route_id in [6036, 6037]:
        retval += 'D Branch'
    elif route_id in [8393, 8394]:
        retval += 'E Branch'
    elif route_id in [6038, 6039]:
        retval += 'D Branch (GLX)'
    elif route_id in [6028, 6029]:
        retval += 'E Branch (GLX)'
    else:
        retval += 'UNKNONWN'
    # end_if 
    return retval
    
def classify_silver_line_route(row):
    # Crude, first-crack implementation based on Ed Bromage's 'Mode'
    # and the TDM19 'Routes_ID' field.
    retval = 'Silver Line - '
    ed_mode = row['Mode']
    route_id = row['Routes_ID']
    if ed_mode == 13:
        if route_id in [8235, 8262]:
            retval += 'SL4'
        elif route_id in [8234, 8263]:
            retval += 'SL5'
        else:
            retval += 'UNKNOWN ' + 'Mode = ' + str(ed_mode) + ' Routes_ID = ' + str(route_id)
    else:
        # ed_mode == 12
        if route_id in [6055, 6056]:
            retval += 'SL1'
        elif route_id in [8256, 8257, 8258, 8259, 8260, 8261]:
            retval += 'SL2'
        elif route_id in [6058, 6059]:
            retval += 'SL3'
        elif route_id in [6050, 6051]:
            retval += 'to Sliver Line Way'
        else:
            retval += 'UNKNOWN ' + 'Mode = ' + str(ed_mode) + ' Routes_ID = ' + str(route_id)
        # end_if
    # end_if  
    return retval
    
def classify_commuter_rail_route(row):
    # Crude, first-crack implementation based on solely Ed Bromage's 'Mode'.
    # This implementation could be moved into the calling logic, but is
    # placed here in expectation that further refinement will be needed.
    # Yes, this version could have been implemented with a lookup table...
    ed_mode = row['Mode']
    retval = 'Commuter Rail - '
    if ed_mode == 9:
        retval += 'Fairmount Line'
    elif ed_mode == 32:
        retval += 'Beverly / Newburyport / Rockport Line'
    elif ed_mode == 33:
        retval += 'Stoughton / Providence Line'
    elif ed_mode == 34:
        retval += 'Greenbush / Plymouth / Kingston / Middleborough Line'
    elif ed_mode == 35:
        retval += 'Haverhill Line'
    elif ed_mode == 36:
        retval +='Lowell Line'
    elif ed_mode == 37:
        retval += 'Fitchburg Line'
    elif ed_mode == 38:
        retval += 'Framingham / Worcester Line'
    elif ed_mode == 39:
        retval += 'Needham Line'
    elif ed_mode == 40:
        retval += 'Franklin Line'
    # end_if
    return retval
# end_def classify_commuter_rail_route()

def classify_transit_route(row):
    retval = 'None'
    eds_mode = row['Mode']
    if eds_mode == 4:
        retval = classify_green_line_route(row)
    elif eds_mode in [12, 13]:
        retval = classify_silver_line_route(row)
    # For the commuter rail lines: for now, just use the brute-force 'Mode' number.
    # elif eds_mode in [9, 34, 35, 36, 37, 38, 39, 40]:
    #    retval = classify_commuter_rail_route(row)
    elif eds_mode in _classification_table:
        retval = _classification_table[eds_mode]
    else:
        retval = 'UNKNOWN'
    # end_if
    return retval
# end_def classify_transit_route()

In [ ]:
# In spite of its name the 'Statewide_Routes' file is in reality a Stops file.
# 
def load_stops_table(scenario):
    temp_df = pd.read_csv(scenario + r'Databases/Statewide_Routes_2018S.csv',
                          usecols=["Mode", "Routes_ID", "Route_Name", "STOP_ID", "STOP_Name"]).drop_duplicates()
    temp_df.rename(columns={"Route_Name": "TransCAD_Route_Name"}, inplace=True)
    temp_df['report_class'] = temp_df.apply(lambda x: classify_transit_route(x), axis=1)
    return temp_df

### End of "library" code

### Here begins the driver logic for this notebook

In [ ]:
# Get total boardings per {route,stop} for each time period ('tod') and for the day as a whole.
TODsums = import_transit_assignment(scenario) 

In [ ]:
if debug_trace_flag:
    TODsums['daily']

In [ ]:
# Create the route classification table
classification_table = load_stops_table(scenario)

In [ ]:
if debug_trace_flag:
    classification_table

In [ ]:
# In order to be able to filter the TODsums data (boarding data) by line or stop,
# Join each of the 'TODsums' dataframes (containing the ONO data) to the stops table
# on TODsums[x].ROUTE <===> stops_table.Routes_ID
for tod in TODsums.keys():
    TODsums[tod] = TODsums[tod].merge(classification_table, 
                                      how='left', left_on='STOP', right_on='STOP_ID')
# 

In [ ]:
if debug_trace_flag:
    TODsums['daily']

In [ ]:
# Extract the 'ONO' data for the light rapid transit lines from the entire blob of ONO data.
#
ono_data = {}
for k in TODsums.keys():
    ono_data[k] = None
#
#
# The following doesn't work - These (simple) strings don't match Mode names in TransCAD! Ugh! :-(
heavy_rail_rt_route_names =  ['Green Line', 'Mattapan Trolley'] 
#
# Warning: Using a QUICK HACK instead!!! 
# Using Ed Bromage's 'Mode' number to filter!!! :-( :-( :-(
light_rail_rt_bromage_mode_ids = [4, 6]
for tod in TODsums.keys():
    boolean_series = TODsums[tod].Mode.isin(light_rail_rt_bromage_mode_ids)
    ono_data[tod] = TODsums[tod][boolean_series]
#

In [ ]:
if debug_trace_flag:
    ono_data['daily']

In [ ]:
## Code beyond this point will need to be changed.

In [ ]:
# Remove columns not to confound the 'groupby' operation --- MAY NOT BE NEEDED
for tod in ono_data.keys():
    ono_data[tod] = \
        ono_data[tod].drop(['ROUTE', 'Routes_ID', 'STOP', 'STOP_ID', 'TransCAD_Route_Name', 'Mode'], axis=1)

In [ ]:
if debug_trace_flag:
    ono_data['daily']

In [ ]:
# Pandas appears to have trouble grouping-by more than one-string valued field.
# We first group-by STOP_Name, and subsequently work back to the "route" name.
grouped_ono_data = {}
for tod in ono_data.keys():
    grouped_ono_data[tod] = None
#
for tod in ono_data.keys():
    grouped_ono_data[tod] = ono_data[tod].groupby(ono_data[tod]['STOP_Name']).sum()
#

In [ ]:
if debug_trace_flag:
    grouped_ono_data['daily']

## Prepare final data frame for output to CSV file.

In [ ]:
am_df = grouped_ono_data['AM']
md_df = grouped_ono_data['MD']
pm_df = grouped_ono_data['PM']
nt_df = grouped_ono_data['NT']
daily_df = grouped_ono_data['daily']

In [ ]:
j1 = pd.merge(am_df, md_df, on=['STOP_Name'], suffixes=('_am', '_md'))

In [ ]:
j2 = pd.merge(pm_df, nt_df, on=['STOP_Name'], suffixes=('_pm', '_nt'))

In [ ]:
j3 = pd.merge(j1, j2, on=['STOP_Name'])

In [ ]:
j4 = pd.merge(daily_df, j3, on=['STOP_Name'])

In [ ]:
# Create STOP_Name to (ersatz) "route name" mapping
# in order to circumvent apparent Pandas issue with group-by on multiple string-valued fields.
temp_df1 = ono_data['daily']
temp_df2 = temp_df1[['STOP_Name', 'report_class']] 
stop_to_route_mapping = temp_df2.drop_duplicates()
if debug_trace_flag:
    stop_to_route_mapping

In [ ]:
if debug_trace_flag:
    dump_csv(stop_to_route_mapping, "stop_to_route_mapping.csv")

In [ ]:
j5 = j4.merge(stop_to_route_mapping, how="left", left_on="STOP_Name", right_on="STOP_Name")

In [ ]:
temp_df = j5.rename(columns={"report_class": "Line"})

In [ ]:
# Order the data frame first by line (then, implicitly, by station.)
red_df = temp_df[temp_df.Line == 'Red Line']
orange_df = temp_df[temp_df.Line == 'Orange Line']
blue_df = temp_df[temp_df.Line == 'Blue Line']
final_df = pd.concat([red_df, orange_df, blue_df])

In [ ]:
final_df.columns

In [ ]:
# Set final order of columns in preparation for writing dataframe out to CSV.
final_column_order = ['Line', 'STOP_Name', 
                      'DirectTransferOff', 'DirectTransferOn', 'DriveAccessOn',
                      'EgressOff', 'Off', 'On', 'WalkAccessOn', 'WalkTransferOff', 'WalkTransferOn', 
                      'DirectTransferOff_am', 'DirectTransferOn_am',
                      'DriveAccessOn_am', 'EgressOff_am', 'Off_am', 'On_am',
                      'WalkAccessOn_am', 'WalkTransferOff_am', 'WalkTransferOn_am',
                      'DirectTransferOff_md', 'DirectTransferOn_md', 'DriveAccessOn_md',
                      'EgressOff_md', 'Off_md', 'On_md', 'WalkAccessOn_md',
                      'WalkTransferOff_md', 'WalkTransferOn_md', 'DirectTransferOff_pm',
                      'DirectTransferOn_pm', 'DriveAccessOn_pm', 'EgressOff_pm', 'Off_pm',
                      'On_pm', 'WalkAccessOn_pm', 'WalkTransferOff_pm', 'WalkTransferOn_pm',
                      'DirectTransferOff_nt', 'DirectTransferOn_nt', 'DriveAccessOn_nt',
                      'EgressOff_nt', 'Off_nt', 'On_nt', 'WalkAccessOn_nt',
                      'WalkTransferOff_nt', 'WalkTransferOn_nt' ]
final_df = final_df[final_column_order]

In [ ]:
if debug_trace_flag:
    final_df

In [ ]:
# Generate fully-qualified name of output CSV file.
#
# First, pluck name of scenario from last element of scenario directory name remove trailing '/'
temp1 = scenario[0:len(scenario)-1]
temp2 = os.path.split(temp1)
# Get 'raw' scenario name: may have blanks. Ugh!
raw_scenario_name = temp2[1]
clean_scenario_name = raw_scenario_name.replace(' ', '_')
#
fq_output_csv_fn = sandbox_dir + clean_scenario_name + '_' + output_csv_fn_suffix + '.csv'

In [ ]:
# Write output CSV report file
#
final_df.to_csv(fq_output_csv_fn, index=False, sep=',')